In [11]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [8]:
def parse_gtf_attributes(attr_string):
    """Parse GTF attribute column into a dictionary."""
    attrs = {}
    for match in re.finditer(r'(\w+) "([^"]+)"', attr_string):
        attrs[match.group(1)] = match.group(2)
    return attrs

def extract_tsl_number(tsl_value):
    """Extract only the number from transcript_support_level."""
    if tsl_value:
        match = re.match(r'^(\d+)', tsl_value)
        if match:
            return match.group(1)
    return 'NA'
def extract_promoters_from_gtf(gtf_file, upstream=250, downstream=250):
    """
    Extract promoter regions (TSS ± window) for protein-coding genes.
    """
    records = []
    
    with open(gtf_file, 'r') as f:
        for line in f:
            if line.startswith('#'):
                continue
            
            fields = line.strip().split('\t')
            feature_type = fields[2]
            
            # Only process transcript entries
            if feature_type != 'transcript':
                continue
            
            attrs = parse_gtf_attributes(fields[8])
            
            # Filter for protein-coding genes
            if attrs.get('gene_biotype') != 'protein_coding':
                continue
            
            chrom = fields[0]
            strand = fields[6]
            
            # Determine TSS based on strand
            if strand == '+':
                tss = int(fields[3])
            else:
                tss = int(fields[4])
            
            # Calculate promoter region
            start = max(1, tss - upstream)
            end = tss + downstream

            # Extract only the number from transcript_support_level
            tsl_raw = attrs.get('transcript_support_level', '')
            tsl_clean = extract_tsl_number(tsl_raw)
            
            records.append({
                'Chrom': chrom,
                'start': start,
                'end': end,
                'transcript': attrs.get('transcript_id', ''),
                'TSS': tss,
                'Gene': attrs.get('gene_name', ''),
                'strand': strand,
                'transcript_support_level': tsl_clean
            })
    
    df = pd.DataFrame(records)
    return df

In [20]:
# Usage
gtf_file = "/mnt/10T-backup-mingju/ifar/RNA_seq/pipeline_dataset/reference/nf_core/Homo_sapiens.GRCh38.113.gtf"
promoter_df = extract_promoters_from_gtf(gtf_file, upstream=250, downstream=250)

print(f"Total protein-coding transcripts: {len(promoter_df)}")

Total protein-coding transcripts: 171867


In [21]:
promoter_df

,Chrom,start,end,transcript,TSS,Gene,strand,transcript_support_level
0,1,3068918,3069418,ENST00000511072,3069168,PRDM16,+,5
1,1,3068933,3069433,ENST00000607632,3069183,PRDM16,+,2
2,1,3068947,3069447,ENST00000378391,3069197,PRDM16,+,1
3,1,3068961,3069461,ENST00000514189,3069211,PRDM16,+,5
4,1,3068953,3069453,ENST00000270722,3069203,PRDM16,+,1
...,...,...,...,...,...,...,...,...
171862,KI270726.1,25991,26491,ENST00000619729,26241,,+,NA
171863,KI270711.1,24400,24900,ENST00000614336,24650,,-,1
171864,KI270711.1,29376,29876,ENST00000612640,29626,,-,1
171865,KI270713.1,35157,35657,ENST00000601199,35407,,+,NA


In [22]:
# Define valid chromosomes
valid_chroms = [str(i) for i in range(1, 23)] + ['X', 'Y']

# Filter dataframe
promoter_df = promoter_df[promoter_df['Chrom'].isin(valid_chroms)]

print(f"Total protein-coding transcripts after filtering: {len(promoter_df)}")
print(promoter_df['Chrom'].unique())

Total protein-coding transcripts after filtering: 171825
['1' '2' '3' '4' '5' '6' '7' 'X' '8' '9' '11' '10' '12' '13' '14' '15'
 '16' '17' '18' '20' '19' 'Y' '22' '21']


In [23]:
promoter_df

,Chrom,start,end,transcript,TSS,Gene,strand,transcript_support_level
0,1,3068918,3069418,ENST00000511072,3069168,PRDM16,+,5
1,1,3068933,3069433,ENST00000607632,3069183,PRDM16,+,2
2,1,3068947,3069447,ENST00000378391,3069197,PRDM16,+,1
3,1,3068961,3069461,ENST00000514189,3069211,PRDM16,+,5
4,1,3068953,3069453,ENST00000270722,3069203,PRDM16,+,1
...,...,...,...,...,...,...,...,...
171820,21,34375354,34375854,ENST00000399295,34375604,SMIM11,+,2
171821,21,34375807,34376307,ENST00000652570,34376057,SMIM11,+,NA
171822,21,34400822,34401322,ENST00000410005,34401072,C21orf140,-,NA
171823,21,34423701,34424201,ENST00000850648,34423951,SMIM34,-,NA


In [32]:
promoter_df = promoter_df[
    promoter_df["Gene"].notna() &
    promoter_df["Gene"].astype(str).str.strip().ne("") &
    ~promoter_df["Gene"].astype(str).str.lower().isin(["nan", "na", "none"])
].copy()

In [33]:
promoter_df

,Chrom,start,end,transcript,TSS,Gene,strand,transcript_support_level
0,1,3068918,3069418,ENST00000511072,3069168,PRDM16,+,5
1,1,3068933,3069433,ENST00000607632,3069183,PRDM16,+,2
2,1,3068947,3069447,ENST00000378391,3069197,PRDM16,+,1
3,1,3068961,3069461,ENST00000514189,3069211,PRDM16,+,5
4,1,3068953,3069453,ENST00000270722,3069203,PRDM16,+,1
...,...,...,...,...,...,...,...,...
171820,21,34375354,34375854,ENST00000399295,34375604,SMIM11,+,2
171821,21,34375807,34376307,ENST00000652570,34376057,SMIM11,+,NA
171822,21,34400822,34401322,ENST00000410005,34401072,C21orf140,-,NA
171823,21,34423701,34424201,ENST00000850648,34423951,SMIM34,-,NA


In [34]:
promoter_df[promoter_df['TSS'] == 6197757]

,Chrom,start,end,transcript,TSS,Gene,strand,transcript_support_level


In [35]:
tsl_counts = promoter_df['transcript_support_level'].value_counts()
print(tsl_counts)

transcript_support_level
1     37586
2     33430
NA    31484
5     28416
3     25490
4     14151
Name: count, dtype: int64


In [36]:
# Save to file
promoter_df.to_csv("/mnt/10T-backup-mingju/ifar/databases/hg38/RegionBased/promoter/promoter_regions_pcg_250bp.csv", sep='\t',header=False,index=False)

In [37]:
! bash ~/PreprocessDB/bed/sorted_lexicographical.sh /mnt/10T-backup-mingju/ifar/databases/hg38/RegionBased/promoter/promoter_regions_pcg_250bp.csv

In [38]:
!awk 'BEGIN{OFS="\t"} { $1="chr"$1; print }' \
/mnt/10T-backup-mingju/ifar/databases/hg38/RegionBased/promoter/promoter_regions_pcg_250bp.csv_sorted_lexicographical \
> /mnt/10T-backup-mingju/ifar/databases/hg38/RegionBased/promoter/promoter_regions_pcg_250bp_sorted.bed
